In [1]:
import os
import pandas as pd
from random import randint
from time import sleep
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from scrapy.http import TextResponse
from tqdm.notebook import tqdm

In [2]:
options = uc.ChromeOptions()
options.add_experimental_option(
    "prefs",
    {
        "download.default_directory": os.getcwd(),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True,
    },
)

options.add_argument("--disable-gpu")
options.add_argument("--disable-popup-blocking")

webdriver = uc.Chrome(options=options)

webdriver.set_page_load_timeout(120)
webdriver.maximize_window()
url = "https://www.startech.com.bd"
webdriver.get(url)

In [3]:
def get_scrapy_res(url, webdriver, xpath):
    response = (
        TextResponse(
            url=url,
            body=webdriver.page_source.encode("utf-8"),
        )
        .xpath(xpath)
        .extract()
    )
    return response

In [4]:
first_level_category_elemnts = webdriver.find_elements(
    By.XPATH, '//ul[@class="navbar-nav"]/li/a'
)
second_level_category_elements = []

for element in first_level_category_elemnts:
    category_name = element.text.strip()
    if category_name.lower() in ["monitor", "phone"]:
        second_level_category_elements.append(element)
        continue

    second_level_category_elements.extend(
        element.find_element(By.XPATH, "..").find_elements(
            By.XPATH,
            './ul/li/a[not(contains(., "Show All")) and not(text() = "All Laptop") and not(text() = "Desktop Offer")] | ./div/ul/li/a[not(contains(., "Show All")) and not(text() = "All Laptop") and not(text() = "Desktop Offer")]',
        )
    )

all_category_links = [
    element.get_attribute("href") for element in second_level_category_elements
]
print(len(all_category_links))

231


In [ ]:
product_category_links = {"category": [], "product_link": []}

for category_link in tqdm(all_category_links):
    webdriver.get(category_link)
    sleep(randint(1, 2))

    category_name = webdriver.find_element(
        By.XPATH,
        '//div[contains(@class, "top-bar")]//label[contains(@class, "page-heading")]',
    ).text.strip()

    select_element = webdriver.find_element(
        By.XPATH,
        '//div[contains(@class, "top-bar")]//div[label[text()="Show:"]]//select',
    )
    last_option_value = select_element.find_elements(By.TAG_NAME, "option")[
        -1
    ].get_attribute("value")
    Select(select_element).select_by_value(last_option_value)
    sleep(2)
    
    
    pagination_counter = 0
    while True:
        print(
            f"Current category: {category_name} | Page link: {webdriver.current_url}",
            flush=True,
            end="\r",
        )

        product_links = get_scrapy_res(
            webdriver.current_url,
            webdriver,
            '//div[contains(@class, "main-content")]//div[div[@class="actions"]/span[contains(., "Buy Now")]]/h4/a/@href',
        )

        for product_link in product_links:
            product_category_links["category"].append(category_name)
            product_category_links["product_link"].append(product_link)
        
        pagination_elements = webdriver.find_elements(By.XPATH, '//ul[@class="pagination"]/li')[2:-1]
        
        if pagination_counter + 1 >= len(pagination_elements):
            break
        
        pagination_counter += 1
        pagination_element = pagination_elements[pagination_counter]
        webdriver.execute_script("arguments[0].scrollIntoView({behavior: 'auto', block: 'center'});", pagination_element)
        sleep(1)
        pagination_element.click()
        sleep(randint(1, 2))

  0%|          | 0/231 [00:00<?, ?it/s]

ElementClickInterceptedException: Message: element click intercepted: Element <li>...</li> is not clickable at point (750, 496). Other element would receive the click: <div class="popup f-in">...</div>
  (Session info: chrome=135.0.7049.52)
Stacktrace:
#0 0x5d539d529e6a <unknown>
#1 0x5d539cfdb640 <unknown>
#2 0x5d539d03399c <unknown>
#3 0x5d539d0317fc <unknown>
#4 0x5d539d02ee92 <unknown>
#5 0x5d539d02e590 <unknown>
#6 0x5d539d020b7d <unknown>
#7 0x5d539d052752 <unknown>
#8 0x5d539d0204fa <unknown>
#9 0x5d539d05291e <unknown>
#10 0x5d539d0787b5 <unknown>
#11 0x5d539d052523 <unknown>
#12 0x5d539d01eef3 <unknown>
#13 0x5d539d01fb51 <unknown>
#14 0x5d539d4eed2b <unknown>
#15 0x5d539d4f2c01 <unknown>
#16 0x5d539d4d6512 <unknown>
#17 0x5d539d4f3774 <unknown>
#18 0x5d539d4ba90f <unknown>
#19 0x5d539d517fb8 <unknown>
#20 0x5d539d518196 <unknown>
#21 0x5d539d528cb6 <unknown>
#22 0x7fe30f29caa4 <unknown>
#23 0x7fe30f329c3c <unknown>


In [ ]:
product_category_links_df = pd.DataFrame(product_category_links)
print(product_category_links_df.shape)
product_category_links_df.head(5)

In [ ]:
product_category_links_df = (
    product_category_links_df.drop_duplicates(subset=["product_link"])
    .reset_index(drop=True)
    .copy()
)
# product_category_links_df.drop(columns=["index"], inplace=True)
print(product_category_links_df.shape)
product_category_links_df.to_csv("./product_category_links.csv", index=False)